# Notes

### Portfolio Optimization

In [9]:
## import packages ##

# solver tools
import scipy
import cvxpy as cp

# basic data and math tools
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas_datareader import data as pdr # for web scraping
import yfinance as yf # tool for download yahoo finance data
import requests # for web scraping data from yahoo finance website if needed 
import re # try different replace string method

# try backtesting library
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pypfopt

#### General Markowitz Portfolio with Practical Constraints

\begin{array}{lll}
\underset{\mathbf{w}}{\textsf{maximize}} & 
\mathbf{w}^{T}\boldsymbol{\mu}-\lambda\mathbf{w}^{T}\boldsymbol{\Sigma}\mathbf{w} & \\
\textsf{subject to}
& \mathbf{w}^{T}\mathbf{1}=1 & \text{budget}\\
& \mathbf{w}\ge\mathbf{0} & \text{no shorting}\\
& \left\Vert \mathbf{w}\right\Vert _{1}\leq\gamma & \text{leverage}\\
& \left\Vert \mathbf{w}-\mathbf{w}_{0}\right\Vert _{1}\leq\tau & \text{turnover}\\
& \left\Vert \mathbf{w}\right\Vert _{\infty}\leq u & \text{max position}\\
& \left\Vert \mathbf{w}\right\Vert _{0}\leq K & \text{sparsity}
\end{array}

#### Markowitz Mean-Variance Portfolio (MVP)

\begin{array}{ll}
\underset{\mathbf{w}}{\textsf{maximize}} & \boldsymbol{\mu}^T\mathbf{w} -\lambda\mathbf{w}^T\mathbf{\Sigma}\mathbf{w}\\
{\textsf{subject to}}
 & \mathbf{1}^T\mathbf{w} = 1\\
 & \mathbf{w}\ge\mathbf{0}.
\end{array}

In [10]:
def MVP(mu, Sigma, lmd=0.5):
    w = cp.Variable(len(mu))
    variance = cp.quad_form(w, Sigma)
    expected_return = w @ mu
    problem = cp.Problem(cp.Minimize(lmd * variance - expected_return), [w >= 0, cp.sum(w) == 1])   
    problem.solve()
    return w.value

#### Global Minimum Variance Portfolio (GMVP)

\begin{array}{ll}
\underset{\mathbf{w}}{\textsf{minimize}} & \mathbf{w}^T\mathbf{\Sigma}\mathbf{w}\\
{\textsf{subject to}}
 & \mathbf{1}^T\mathbf{w} = 1\\
 & \mathbf{w}\ge\mathbf{0}
\end{array}

In [11]:
# function for GMVP
def GMVP(Sigma):
     w = cp.Variable(np.shape(Sigma)[0])
     variance = cp.quad_form(w, Sigma)
     problem = cp.Problem(cp.Minimize(variance),
                    [w >= 0, cp.sum(w) == 1])
     problem.solve()
     return w.value

#### Maximum Sharpe Ratio Portfolio (MSRP)

Nonconvex Form: 
\begin{array}{ll}
\underset{\mathbf{w}}{\textsf{maximize}} &
\dfrac{\mathbf{w}^{T}\boldsymbol{\mu}-r_{f}}{\sqrt{\mathbf{w}^{T}\boldsymbol{\Sigma}\mathbf{w}}}\\
\textsf{subject to} & \mathbf{1}^{T}\mathbf{w}=1\\
& \mathbf{w}\ge\mathbf{0}
\end{array}

Rewrite to convex form via schaible:
\begin{array}{ll}
\underset{\tilde{\mathbf{w}}}{\textsf{minimize}} & \tilde{\mathbf{w}}^T\mathbf{\Sigma}\tilde{\mathbf{w}}\\
{\textsf{subject to}}
 & \tilde{\mathbf{w}}^T\boldsymbol{\mu} = 1\\
 & \tilde{\mathbf{w}}\ge\mathbf{0}
\end{array}

In [12]:
def MSRP(mu, Sigma):
    w = cp.Variable(len(mu))
    variance = cp.quad_form(w, Sigma)
    problem = cp.Problem(cp.Minimize(variance), [w >= 0, w @ mu == 1])   
    problem.solve()
    return w.value/np.sum(w.value)